<a href="https://colab.research.google.com/github/muziejus/21F-UP206A/blob/master/2025/march_5_data_engineering_with_polars_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A blue background with the pandas logo and the words Columbia Data Club on it](https://raw.githubusercontent.com/columbia-data-club/meetings/main/assets/images/2025/polars.png)

# Python Data Engineering with Polars I

March 5, 2025

by [Moacir P. de Sá Pereira](https://moacir.com) for the [Columbia Data Club](https://github.com/columbia-data-club/)


This notebook provides an introduction to data engineering with [Polars](https://pola.rs). A basic understanding of Python syntax (such as the one covered in the Data Club’s [Intro to Python video](https://youtu.be/l45rzo4MUHs)) should suffice.

We will be looking to our perennial favorite today, [NYC Yellow Cab trip data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

## Pandas, OK, but Polars?

[Polars](https://pola.rs) is a library that has a lot of similar use cases to pandas. The developers present the library as “DataFrames for a new era,” and they mention a few specific reasons why it would make sense to switch to Polars, especially for intensive data engineering (with large datasets).

* Written from the ground up in Rust, so the code is very close to the machine
* This means blazing fast performance, even in comparison to PySpark and Dask, leaving pandas in its dust.
* Query optimization by using specific Polars expressions (similar to Spark)
* Lazy loading

Pandas still has a lot of use cases, of course, and probably is the better choice for row-wise operations that use methods like `.itterrows()`, but within the context of [ETL](https://en.wikipedia.org/wiki/Extract,_transform,_load), where we are transforming large datasets, we want to leverage certain optimizations that are built into Polars’s memory model.

In [ ]:
import polars as pl
from datetime import datetime as dt

## Polars Syntax

Polars is built on two fundamental concepts in terms of its data wrangling engine, **contexts** and **expressions**. The latter are lazy descriptions of data transformations that make use of methods and functions built into Polars.

To me, an expression is a bit like a lambda function, down to how you can even predefine it as a transformation before you have even executed it. Here we calculate a what percentage of a taxi fare was paid as the tip:

In [ ]:
tip_pct = pl.col("tip_amount") / pl.col("fare_amount")
tip_pct

The `pl.col()` method is referencing a specific column in a dataset, but no calculations have been done yet. In fact, we have not even loaded data.

In [ ]:
df = pl.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-12.parquet")

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.select(passengers=pl.col("passenger_count").value_counts(sort=True))#.unnest("passengers")

What do these commands demonstrate to you?

## Polars Context

We have our one example of a Polars expression, and we’ll look at others in a bit, but recall that the expression needs a context. The four main contexts are:

* `select`
* `with_columns`
* `filter`
* `group_by`

Their behaviour should be somewhat familiar.

### `select`

`select` lets you choose but also aggregate certain columns.

In [ ]:
df.select(
    trip_duration = pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"),
    distance = pl.col("trip_distance")).head()

In [ ]:
df.select(
    trip_duration = pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"),
    distance = pl.col("trip_distance")).describe()

In [ ]:
df.select(
    distance = pl.col("trip_distance"),
    tip_pct = tip_pct,
    mean_tip_pct = tip_pct.mean()
).head()


In [ ]:
df.select(
    tip_pct = tip_pct,
    distance = pl.col("trip_distance"),
    mean_tip_pct = tip_pct.mean()
).head()

In [ ]:
df.select(tip_pct = tip_pct).describe()

In [ ]:
df.select(pl.col("fare_amount")).describe()

In [ ]:
df.filter(pl.col("fare_amount")== 0)

In [ ]:
df.select(fare_amount=pl.col("fare_amount")+0.001).describe()

### `with_columns`

We can see that the `mean_tip_pct` does not get calculated correctly because the maximum tip percentage is infinity, because some people had \$0 fares. We can use `with_columns` to fix this by temporarily adding a tenth of a cent to each fare, so no fares are \$0.

`with_columns` works like `select` but returns a new data frame with all the columns as well as the newly added ones.

In [ ]:
df.with_columns(
    fare_amount=pl.col("fare_amount")+0.001
).select(
    "fare_amount",
    tip_pct = tip_pct,
    distance = pl.col("trip_distance"),
    mean_tip_pct = tip_pct.mean()
).head()

In [ ]:
df_with_tip_pct = df.with_columns(
    fare_amount=pl.col("fare_amount")+0.001,
    tip_pct = tip_pct
)

In [ ]:
df_with_tip_pct.head()

### `filter`

Alternatively, we could filter out the $0 fares or only adjust them using the `filter`:

In [ ]:
df.filter(pl.col("fare_amount") != 0).select(
    tip_pct = tip_pct,
    distance = pl.col("trip_distance"),
    mean_tip_pct = tip_pct.mean()
).head()

Or use a new expression, `when`:

In [ ]:
df.with_columns(
    fare_amount=pl.when(
        pl.col("fare_amount") == 0
    ).then(
        pl.col("fare_amount")+0.001
    ).otherwise(pl.col("fare_amount")
    )
).select(
    "fare_amount",
    tip_pct = tip_pct,
    distance = pl.col("trip_distance"),
    mean_tip_pct = tip_pct.mean()
).head()

We can filter by date.

In [ ]:
df.filter(
    pl.col("tpep_pickup_datetime").cast(pl.Date) == dt(2024, 12, 25)
).select(
    "tpep_pickup_datetime",
    pickup_time = pl.col("tpep_pickup_datetime").cast(pl.Time),
    distance = pl.col("trip_distance"),
).head()



In [ ]:
df.filter(
    pl.col("tpep_pickup_datetime").is_between(
        dt(2024, 12, 25, 23, 58),
        dt(2024, 12, 26, 0, 1)
    )
).select(
    "tpep_pickup_datetime",
    distance = pl.col("trip_distance"),
    trip_count = pl.len(),
    mean_trip_distance = pl.col("trip_distance").mean(),
    max_trip_distance = pl.col("trip_distance").max(),
    min_trip_distance = pl.col("trip_distance").min(),
).head()

### `group_by`

Finally, we can use grouping to group our data.

In [ ]:
df.group_by(pl.col("passenger_count")).agg(
  trip_count=pl.len(),
  mean_trip_distance = pl.col("trip_distance").mean(),
  mean_fare_amount = pl.col("fare_amount").mean().round(2),
).sort("passenger_count")

In [ ]:
df.sort("tpep_pickup_datetime").group_by_dynamic("tpep_pickup_datetime", every="1d").agg(
    trip_count=pl.len(),
).with_columns(
    month=pl.col("tpep_pickup_datetime").dt.month(),
    day=pl.col("tpep_pickup_datetime").dt.day(),
)

## We Are Low-Key EDAing Here?

Absolutely. We are using these contexts to help us understand our data better, with an eye toward wrangling it for future analysis.

What are some assumptions about our data we should test and perhaps correct?